In [1]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

# Going through the example provided in:
# https://brightdata.com/blog/how-tos/web-scraping-with-python

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

In [2]:
browser = webdriver.Firefox()
url = 'https://cs.deals/fi/market/csgo/?sort=discount&sort_desc=1'
browser.get(url)
browser.implicitly_wait(0.5)
time.sleep(10)
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")
courts_list = soup.find_all("div", class_ = "item csgo")
print(courts_list)

[<div class="item csgo" style="border-bottom: 2px solid rgb(235, 75, 75);"><div class="top"><div class="name">Chrome Cannon</div><div class="subName">Minimal Wear</div><div class="info"><div class="more"><span class="val"></span><i aria-hidden="true" class="far fa-times"></i></div></div><div class="pic"><img alt="AWP | Chrome Cannon (Minimal Wear)" class="itemimg" loading="lazy" src="/steamItems/MTU0MDMw_2.png"/><div class="type">AWP</div></div></div><div class="stickers"></div><img alt="CSGO" class="appIcon" src="/images/steam/appicons/730.png"/><div class="price">94.16 €<span class="discount">-36.06%</span></div><div class="floatBar" style="display: none;"><i aria-hidden="true" class="fas fa-caret-down"></i></div><div class="locktime"><i aria-hidden="true" class="fal fa-lock"></i> 6 päivää</div><div class="addToCartWrapper"><div class="addToCartBtnWrapper"><span class="addToCartBtn">Lisää koriin</span></div><div class="menuBtnWrapper"><i aria-hidden="true" class="far fa-bars"></i></d

In [3]:
# Create a list of 100 first items ordered by discount
# [name, price, discount]
complete_list = []
str_courts_list = str(courts_list)

# Add item prices to the list
prices_list = re.findall("(price\">|small\">)(.*?)€", str_courts_list)
for j,i in prices_list:
    i_changed = i.replace(",","")
    i_changed2 = i_changed.replace(" ", "")
    if len(complete_list) < 100:
        complete_list.append([0,i_changed2,0])

# Add item names to the list
names_list = re.findall("<img alt=\"(.*?)\"", str_courts_list)
for word in names_list:
    if word == "CSGO":
        names_list.remove(word)
    elif word == "sticker":
        names_list.remove(word)
for name in range(100):
    complete_list[name][0] = names_list[name]

# Add item discounts to the list
discounts_list = re.findall("\"discount\">(.*?)</span>", str_courts_list)
for discount in range(100):
    complete_list[discount][2] = discounts_list[discount]

for row in complete_list:
    print(row)


['AWP | Chrome Cannon (Minimal Wear)', '94.16', '-36.06%']
['★ Moto Gloves | Eclipse (Field-Tested)', '126.78', '-29.58%']
['★ Moto Gloves | Finish Line (Battle-Scarred)', '70.77', '-28.66%']
['★ StatTrak™ Ursus Knife | Blue Steel (Field-Tested)', '169.07', '-26.36%']
['★ Butterfly Knife | Marble Fade (Factory New)', '1825.31', '-25.91%']
['Kilowatt Case', '1.67', '-23.74%']
['★ Karambit | Lore (Factory New)', '2087.57', '-23.07%']
['Sir Bloody Loudmouth Darryl | The Professionals', '34.47', '-23.02%']
['★ StatTrak™ Bowie Knife | Night (Field-Tested)', '120.46', '-22.64%']
['Kilowatt Case', '1.71', '-21.92%']
['AWP | Asiimov (Field-Tested)', '115.82', '-20.89%']
['★ Bowie Knife | Black Laminate (Factory New)', '166.78', '-19.55%']
['Operation Breakout Weapon Case', '5.44', '-18.68%']
['★ Paracord Knife | Urban Masked (Battle-Scarred)', '87.37', '-16.04%']
['StatTrak™ USP-S | Jawbreaker (Field-Tested)', '28.44', '-15.81%']
['Operation Breakout Weapon Case', '5.75', '-14.05%']
['Five-Sev